In [ ]:
from itertools import product, combinations
import numpy as np
import networkx as nx
from icecream import ic
import heapq
import matplotlib.pyplot as plt
import random
from collections import deque

In [ ]:
def create_problem(
    size: int,
    *,
    density: float = 1.0,
    negative_values: bool = False,
    noise_level: float = 0.0,
    seed: int = 42,
) -> np.ndarray:
    """Problem generator for Lab3"""
    rng = np.random.default_rng(seed)
    map = rng.random(size=(size, 2))
    problem = rng.random((size, size))
    if negative_values:
        problem = problem * 2 - 1
    problem *= noise_level
    for a, b in product(range(size), repeat=2):
        if rng.random() < density:
            problem[a, b] += np.sqrt(
                np.square(map[a, 0] - map[b, 0]) + np.square(map[a, 1] - map[b, 1])
            )
        else:
            problem[a, b] = np.inf
    np.fill_diagonal(problem, 0)
    return (problem * 1_000).round(), map

In [ ]:
problem, map = create_problem(100, density=0.2, noise_level=0.1, negative_values=False)

In [ ]:
masked = np.ma.masked_array(problem, mask=np.isinf(problem))
G = nx.from_numpy_array(masked, create_using=nx.DiGraph)

In [ ]:
for s, d in combinations(range(problem.shape[0]), 2):
    try:
        # path = nx.shortest_path(G, s, d, weight='weight')
        path = nx.bellman_ford_path(G, s, d, weight='weight')
        cost = cost = nx.path_weight(G, path, weight='weight')
    except nx.NetworkXNoPath:
        # Nodes are not connected
        path = None
        cost = np.inf
    except nx.NetworkXUnbounded:
        # Negative cycle detected
        path = None
        cost = -np.inf
    ic(s, d, path, cost)
None

In [ ]:
def plot_path(G, map, path):
    plt.figure(figsize=(6, 6))
    # draw all edges
    for u, v, data in G.edges(data=True):
        w = data["weight"]
        if not np.isinf(w):
            plt.plot([map[u,0], map[v,0]],
                     [map[u,1], map[v,1]],
                     color="gray", alpha=0.4)
    # highlight shortest path
    if path:
        for i in range(len(path)-1):
            a, b = path[i], path[i+1]
            plt.plot([map[a,0], map[b,0]],
                     [map[a,1], map[b,1]],
                     color="red", linewidth=3)
    # draw nodes
    plt.scatter(map[:,0], map[:,1], s=20, color="gray", edgecolor="black")
    # labels
    for i in range(len(map)):
        plt.text(map[i,0], map[i,1] + 0.015, str(i), ha="center")
    plt.axis("off")
    plt.title("Shortest Path")
    plt.show()

# print("PATH:", path)
# print("COST:", cost)
# plot_path(G, map, path)

In [ ]:
"""
def h(n, goal, map):
    # Heuristic: euclidean distance between node n and goal
    return np.sqrt((map[n,0] - map[goal,0])**2 + (map[n,1] - map[goal,1])**2)
"""

def h(n, goal, map):
    # heuristic: euclidean distance between node n and goal (faster with numpy)
    dx = map[n, 0] - map[goal, 0]
    dy = map[n, 1] - map[goal, 1]
    return np.hypot(dx, dy)

def a_star(G, start, goal, map):
    # --- Check negative weights ---
    for u, v, data in G.edges(data=True):
        w = data.get("weight", float("inf"))
        if w < 0:
            raise ValueError(f"A*: negative values detected ({u} → {v}) con peso {w}")
    # ------------------------------
    nodes = list(G.nodes)
    n_nodes = len(nodes)
    idx = {node: i for i, node in enumerate(nodes)}
    s = idx[start]
    d = idx[goal]
    # cost g from start to node
    g = np.full(n_nodes, np.inf)
    # estimated total cost (g + h)
    f = np.full(n_nodes, np.inf)
    # to build the final solution
    parent = np.full(n_nodes, -1, dtype=int)
    visited = np.zeros(n_nodes, dtype=bool)
    g[s] = 0.0
    f[s] = h(start, goal, map)
    # priority queue (f-value, node-index)
    pq = [(f[s], s)] 
    while pq:
        _, u = heapq.heappop(pq) # pop min f
        if visited[u]:
            continue
        visited[u] = True
        if u == d: # destination (goal) reached 
            break
        # explore neighbors
        for neighbor, data in G[nodes[u]].items():
            v = idx[neighbor]
            w = data.get("weight", float("inf"))
            if np.isinf(w): # no edge, skip 
                continue
            new_cost = g[u] + w
            if new_cost < g[v]:
                g[v] = new_cost
                f[v] = new_cost + h(neighbor, goal, map)
                parent[v] = u
                heapq.heappush(pq, (f[v], v))
    # if not reachable
    if np.isinf(g[d]):
        return None, np.inf
    # reconstruct path
    path = []
    u = d
    while u != -1:
        path.append(nodes[u])
        u = parent[u]
    return path[::-1], g[d]

In [ ]:
def build_neighbors(matrix):
    # pre-computation
    return [np.where(row != np.inf)[0] for row in matrix]

def path_cost(path, matrix):
    """compute the total cost of a path by summing the weight of the edges"""
    if len(path) < 2:
        return 0
    p = np.array(path)
    return matrix[p[:-1], p[1:]].sum()

def greedy_path(start, goal, matrix, neighbors):
    """start from basic greedy search: always expand the cheapest current path found so far"""
    pq = [(0.0, start)]
    parent = {start: None}
    dist = {start: 0.0}
    while pq:
        cost, u = heapq.heappop(pq)
        if u == goal: # destination reached, reconstruct path
            path = []
            while u is not None:
                path.append(u)
                u = parent[u]
            return path[::-1]
        # explore neighborhood
        for v in neighbors[u]:
            new_cost = cost + matrix[u, v]
            if v not in dist or new_cost < dist[v]:
                dist[v] = new_cost
                parent[v] = u
                heapq.heappush(pq, (new_cost, v))
    return None # path not found

def local_search(path, matrix, max_iters=50):
    """try to locally improve the path by replacing intermediate nodes with some alternatives"""
    if path is None:
        return None
    # pre-computation
    current_cost = path_cost(path, matrix) 
    path_set = set(path)
    n_nodes = matrix.shape[0]
    improved = True
    it = 0
    while improved and it < max_iters:
        improved = False
        it += 1
        n = len(path)
        for i in range(1, n - 2):
            prev = path[i - 1]
            curr = path[i]
            nxt = path[i + 1]
            # try some random candidate replacements for this node
            candidates = list(range(n_nodes))
            random.shuffle(candidates)
            for alt in candidates[:30]:  
                if alt in path_set: # no cycles
                    continue
                # the alternative must connect to prev and next node
                if matrix[prev, alt] == np.inf or matrix[alt, nxt] == np.inf:
                    continue
                # compute new cost
                new_cost = (
                    current_cost
                    - matrix[prev, curr] - matrix[curr, nxt]
                    + matrix[prev, alt]  + matrix[alt, nxt]
                )
                if new_cost < current_cost:
                    path_set.remove(curr)
                    path_set.add(alt)
                    path[i] = alt
                    current_cost = new_cost
                    improved = True
                    break
            if improved:
                break
    return path

def is_valid_path(path, matrix):
    for a, b in zip(path, path[1:]):
        if matrix[a, b] == np.inf:
            return False
    return True

def generate_neighbors(path, matrix, neighbors, num_neighbors=25):
    """generate alternative paths (neighbors) by cutting the path and regrowing it"""
    L = len(path)
    if L < 4: # safe cut
        return []
    results = []
    used = set() # avoid duplicates
    goal = path[-1]
    for _ in range(num_neighbors):
        cut_max = max(1, L - 4) 
        i = random.randint(1, cut_max)
        start_suffix = path[i]
        # cut and generate next suffix
        new_suffix = greedy_path(start_suffix, goal, matrix, neighbors)
        if new_suffix is None:
            continue
        prefix = path[:i]
        prefix_set = set(prefix)
        cleaned_suffix = []
        for node in new_suffix:
            if node not in prefix_set:
                cleaned_suffix.append(node)
                prefix_set.add(node)
        if len(cleaned_suffix) < 2:
            continue
        candidate = prefix + cleaned_suffix
        if not is_valid_path(candidate, matrix):
            continue
        key = tuple(candidate)
        if key in used:
            continue
        used.add(key)
        results.append(candidate)
    return results

def tabu_shortest_path(matrix, start, goal,
                       iterations=200,
                       tabu_size=50,
                       neighborhood_size=25):
    # pre-computation
    matrix_neighbors = build_neighbors(matrix)
    # build the initial solution
    base = greedy_path(start, goal, matrix, matrix_neighbors)
    if base is None:
        return None, np.inf
    # local improvement of the initial path
    current = local_search(base, matrix)
    best = current[:]
    best_cost = path_cost(best, matrix)
    # tabu list to avoid revisiting recent solutions
    def key_of(path):
        return hash(tuple(path))
    tabu_list = deque(maxlen=tabu_size)   
    tabu_set = set() # check O(1)
    k = key_of(current)
    tabu_list.append(k)
    tabu_set.add(k)

    for _ in range(iterations):
        # generate neighborhood
        neighbors = generate_neighbors(current, matrix, matrix_neighbors, num_neighbors=neighborhood_size)
        if not neighbors:
            break 
        # improve each neighbor locally and compute its cost (or maybe not, might be too slow)
        scored_neighbors = []
        for p in neighbors:
            # p = local_search(p, matrix, max_iters=10) even more search but slow for large instances
            c = path_cost(p, matrix)
            scored_neighbors.append((c, p))
        # sort neighbors by cost (best first)
        scored_neighbors.sort(key=lambda x: x[0])

        chosen = None
        for cost, candidate in scored_neighbors:
            k = hash(tuple(candidate))
            if k not in tabu_set or cost < best_cost: # found best improvement
                improved = local_search(candidate[:], matrix, max_iters=10) # improve only here
                ci = path_cost(improved, matrix)
                chosen = (improved, ci)
                break
        if chosen is None: # all neighbors were tabu and none improved the best solution -> stop
            break
        candidate, cost = chosen
        current = candidate
        # update tabu list
        k = hash(tuple(current))
        if k not in tabu_set:
            tabu_list.append(k)
            tabu_set.add(k)
            if len(tabu_list) > tabu_size:
                old = tabu_list.popleft()
                tabu_set.discard(old)
        # update global best solution
        if cost < best_cost:
            best = current[:]
            best_cost = cost
    return best, best_cost

In [ ]:
for s, d in combinations(range(problem.shape[0]), 2):
    path, cost = tabu_shortest_path(problem, s, d)
    ic(s, d, path, cost)
None

In [ ]:
for s, d in combinations(range(problem.shape[0]), 2):
    # ! not supposed to work with negative weights -> raise error
    path, cost = a_star(G, s, d, map)
    ic(s, d, path, cost)
None